In [ ]:
# Import Relevant Outliers for the Work Required
import sklearn.preprocessing as prep
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
# Load up datafram and test data
df_test = pd.read_csv(r'test_pub.csv')
df = pd.read_csv(r'train.csv').sample(frac=1)

In [2]:
###### PREPROCESSING ######

In [3]:
# Example code from Jun that turns all classifications into binaries
# This will remove any unknown entries and assist with using the datas to our advantage
df_onehot = pd.get_dummies(df)

keys = df_onehot.keys()
data_keys = [k for k in keys
    if '?' not in k and k[-3:] != "50K"]
data_train = df_onehot[data_keys]
target_train = df_onehot["Salary_ >50K"]

df_onehot1 = pd.get_dummies(df_test)
# add all zero to non-existing keys
for k in data_keys:
    if k not in df_onehot1.keys():
        df_onehot1[k] = 0

data_test = df_onehot1[data_keys]

NameError: name 'pd' is not defined

In [46]:
# Trim Features Used in Testing
data_train_features = [i for i in data_train.keys()]

# Remove Final Weight and ID, they may be used at a later stage but are being ignored for training
# data_train_features.remove('weight')
data_train_features.remove('ID')

# Since binarisation blows the native country category out we will be keeping only the American natives for training
native_keys = [i for i in data_train.keys() if 'Native' in i]
native_keys.remove('Native country_ United-States')
data_train_features = [i for i in data_train_features if i not in native_keys]

In [64]:
# Look at the descrete functions for data preprocessing for normalisation
max_abs_scalar = prep.MaxAbsScaler()
min_max_scalar = prep.MinMaxScaler()
# normalize = prep.normalize(df)
# Normalise Work hours per week
# Normalise Capital Gain / Capital Loss

col_names = ['Work hours per week','Age','Education years','Capital gain','Capital loss']

scaled_features = data_train.copy()
features = scaled_features[col_names]
scaler = prep.normalize(features.values)
# features = scaler.transform(features.values)
data_train[col_names] = features

C:\Users\chubb\Anaconda3\envs\Data Anlaytics\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([[1.66579189e-01, 4.56370467e-06, 9.86028078e-01, ...,
        1.78579748e-05, 0.00000000e+00, 0.00000000e+00],
       [1.19207470e-01, 1.76878804e-06, 9.92869367e-01, ...,
        4.42197009e-06, 0.00000000e+00, 0.00000000e+00],
       [1.40164609e-01, 2.08269770e-06, 9.90128215e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.38572597e-02, 1.49032888e-06, 9.99037807e-01, ...,
        3.11929301e-06, 0.00000000e+00, 0.00000000e+00],
       [3.50419760e-02, 2.00811115e-06, 9.99385841e-01, ...,
        3.92891311e-06, 0.00000000e+00, 0.00000000e+00],
       [1.32495115e-02, 3.66362823e-06, 9.99912221e-01, ...,
        5.49544235e-06, 0.00000000e+00, 0.00000000e+00]])

In [65]:
data_train.describe()

,ID,Age,weight,Education years,Capital gain,Capital loss,Work hours per week,Employment class_ Federal-gov,Employment class_ Local-gov,Employment class_ Never-worked,...,Native country_ Portugal,Native country_ Puerto-Rico,Native country_ Scotland,Native country_ South,Native country_ Taiwan,Native country_ Thailand,Native country_ Trinadad&Tobago,Native country_ United-States,Native country_ Vietnam,Native country_ Yugoslavia
count,38000.000000,38000.000000,3.800000e+04,38000.000000,38000.000000,38000.000000,38000.000000,38000.000000,38000.000000,38000.000000,...,38000.000000,38000.000000,38000.000000,38000.000000,38000.000000,38000.000000,38000.000000,38000.000000,38000.000000,38000.000000
mean,23987.651500,0.428063,1.893863e+05,0.629365,0.010981,0.020157,0.408919,0.030184,0.065737,0.000211,...,0.000947,0.003737,0.000447,0.002447,0.001737,0.000500,0.000500,0.897132,0.002342,0.000500
std,13840.226031,0.151622,1.052969e+05,0.159609,0.074783,0.092728,0.124715,0.171096,0.247825,0.014508,...,0.030765,0.061016,0.021147,0.049411,0.041640,0.022355,0.022355,0.303791,0.048339,0.022355
min,1.000000,0.188889,1.228500e+04,0.062500,0.000000,0.000000,0.010101,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12012.750000,0.311111,1.177460e+05,0.562500,0.000000,0.000000,0.404040,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,24003.500000,0.411111,1.783700e+05,0.625000,0.000000,0.000000,0.404040,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,35947.250000,0.522222,2.363280e+05,0.750000,0.000000,0.000000,0.454545,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,48000.000000,1.000000,1.455435e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
###### MODEL TRAINING ######

In [8]:
### Random Forest Classifier ###

In [66]:
# Intialise Data and function
from sklearn.ensemble import RandomForestClassifier as rdmfrst

feature_list = data_train_features
# feature_list = best_feature_scores
# fearure_list = [i for i in data_train_features if 'Education leve' in i] # since education did not make the best feats list

# recombine data for shuffling
forest_data = data_train[feature_list]

# split target now that it is shuffled
forest_target = target_train

n_training_samples = int(len(forest_data) *.80)
n_validation_samples = len(forest_data) - n_training_samples

forest_train = forest_data.head(n_training_samples)
forest_train_target = forest_target.head(n_training_samples)
forest_validation = forest_data.tail(n_validation_samples)
forest_validation_target = forest_target.tail(n_validation_samples)

In [94]:
# Build Classifier
clf = rdmfrst()
# clf = rdmfrst(**frst_best_settings)

In [95]:
# Build Model
clf.fit(X=forest_train,y=forest_train_target)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [69]:
# Validate Model
forest_validation_probabilities = clf.predict_proba(forest_validation)[:,1]
roc_auc_score(y_true=forest_validation_target, y_score=forest_validation_probabilities)

0.9494884535230397

In [18]:
## Random Forest Parameter Tuning ##
from sklearn.model_selection import RandomizedSearchCV

distributions = {
    'n_estimators': list(range(50,3001,50)),
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(1,30)),
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True,False],
    'class_weight': [None,'balanced', 'balanced_subsample'],
}
tuning_clf = RandomizedSearchCV(rdmfrst(), distributions, random_state=0)
search = tuning_clf.fit(X=forest_train,y=forest_train_target)
frst_best_settings = search.best_params_

In [82]:
## Random Tree Feature Selection ##
# Return the key sorted by feature importance
def return_feature_importance(keys,clf):
    key_importance = []
    for i in range(len(keys)):
        key_importance += [(keys[i], clf.feature_importances_[i])]
    return sorted(key_importance, key=lambda a: a[1], reverse=True)
    

In [99]:
feature_list = [i[0] for i in return_feature_importance(feature_list,clf)[:20]]
best_feature_scores = []

Combinations: 364.0
('Relationship status_ Husband', 'Marital status_ Never-married', 'Occupation_ Exec-managerial') : 0.9352724024667751
('Relationship status_ Husband', 'Occupation_ Exec-managerial', 'Occupation_ Prof-specialty') : 0.9361939623865901
('Relationship status_ Husband', 'Occupation_ Exec-managerial', 'Employment class_ Private') : 0.9384896686541747
Completed 50 of 364.0
('Marital status_ Never-married', 'Occupation_ Exec-managerial', 'Employment class_ Private') : 0.9389324835366148
Completed 100 of 364.0
('Occupation_ Exec-managerial', 'Occupation_ Prof-specialty', 'Employment class_ Private') : 0.9407788504426114
Completed 150 of 364.0
('Occupation_ Exec-managerial', 'Employment class_ Private', 'Sex_ Female') : 0.9411236464057615
Completed 200 of 364.0
Completed 250 of 364.0
Completed 300 of 364.0
Completed 350 of 364.0
('Occupation_ Exec-managerial', 'Employment class_ Private', 'Sex_ Female')
0.9411236464057615


In [23]:
# Output
forest_test_probabilities = clf.predict_proba(data_test[feature_list])[:,1]
df_test['Predicted'] = forest_test_probabilities
df_test[["ID","Predicted"]].to_csv("random_forest_v2.csv", index=False)

In [105]:
### Support Vector Classifier ###

In [105]:
from sklearn.neighbors import KNeighborsClassifier as knn

# feature_list = data_train_features
fearure_list = best_feature_scores
# fearure_list = [i for i in data_train_features if 'Education leve' in i] # since education did not make the best feats list

# recombine data for shuffling
knn_data = data_train[feature_list]

# split target now that it is shuffled
knn_target = target_train

n_training_samples = int(len(knn_data) *.70)
n_validation_samples = len(knn_data) - n_training_samples

knn_train = knn_data.head(n_training_samples)
knn_train_target = knn_target.head(n_training_samples)
knn_validation = knn_data.tail(n_validation_samples)
knn_validation_target = knn_target.tail(n_validation_samples)

In [107]:
# clf = knn(n_neighbors=2)
clf = knn(**knn_best_settings)

In [108]:
clf.fit(X=knn_train,y=knn_train_target)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=101, p=2,
                     weights='distance')

In [109]:
knn_validation_probabilities = clf.predict_proba(knn_validation)[:,1]
roc_auc_score(y_true=knn_validation_target, y_score=knn_validation_probabilities)

0.9475245188329601

In [106]:
## K Nearest Neighbour Parameter Tuning ##
from sklearn.model_selection import RandomizedSearchCV
import math

distributions = {
    'n_neighbors': list(range(1,int(math.sqrt(len(knn_train))),50)),
    'weights':['uniform', 'distance'],
    'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']
}
tuning_clf = RandomizedSearchCV(knn(), distributions, random_state=0)
search = tuning_clf.fit(X=knn_train,y=knn_train_target)
knn_best_settings = search.best_params_

In [110]:
knn_best_settings

{'weights': 'distance', 'n_neighbors': 101, 'algorithm': 'ball_tree'}

In [39]:
# Output
knn_test_probabilities = clf.predict_proba(data_test[feature_list])[:,1]
df_test['Predicted'] = knn_test_probabilities
df_test[["ID","Predicted"]].to_csv("k_nearest_neighbour_v2.csv", index=False)

In [20]:
### Logistic Regression Classifier ###

In [75]:
from sklearn.linear_model import LogisticRegression as lr

# feature_list = data_train_features
fearure_list = best_feature_scores
# fearure_list = [i for i in data_train_features if 'Education leve' in i] # since education did not make the best feats list

# recombine data for shuffling
lr_data = data_train[feature_list]

# split target now that it is shuffled
lr_target = target_train

n_training_samples = int(len(lr_data) *.70)
n_validation_samples = len(lr_data) - n_training_samples

lr_train = lr_data.head(n_training_samples)
lr_train_target = lr_target.head(n_training_samples)
lr_validation = lr_data.tail(n_validation_samples)
lr_validation_target = lr_target.tail(n_validation_samples)

In [76]:
# clf = lr(random_state=0,max_iter=2000)
clf = lr(**lr_best_settings)

In [77]:
clf.fit(X=lr_train, y=lr_train_target)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=3000, multi_class='auto', n_jobs=None,
                   penalty='none', random_state=None, solver='saga', tol=0.0001,
                   verbose=0, warm_start=False)

In [59]:
lr_validation_probabilities = clf.predict_proba(lr_validation)[:,1]
roc_auc_score(y_true=lr_validation_target, y_score=lr_validation_probabilities)

0.9038374922791135

In [74]:
## Logistic Regression Parameter Tuning ##
from sklearn.model_selection import RandomizedSearchCV

distributions = {
    'penalty':['l1','l2', 'elasticnet', 'none'],
    'dual': [True, False],
    'class_weight': [None, 'balanced'],
    'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': list(range(2000,10000,1000)),
}
tuning_clf = RandomizedSearchCV(lr(), distributions, random_state=0)
search = tuning_clf.fit(X=lr_train,y=lr_train_target)
lr_best_settings = search.best_params_

C:\Users\chubb\Anaconda3\envs\Data Anlaytics\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got elasticnet penalty.


C:\Users\chubb\Anaconda3\envs\Data Anlaytics\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver newton-cg supports only dual=False, got dual=True


C:\Users\chubb\Anaconda3\envs\Data Anlaytics\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet penalty.


C:\Users\chubb\Anaconda3\envs

In [111]:
# Output
lr_test_probabilities = clf.predict_proba(data_test[feature_list])[:,1]
df_test['Predicted'] = lr_test_probabilities
df_test[["ID","Predicted"]].to_csv("logistic_regression_v2.csv", index=False)

In [40]:
#### Combining Predictions ####
df_test['Predicted'] = (lr_test_probabilities + knn_test_probabilities + forest_test_probabilities) / 3
df_test[["ID","Predicted"]].to_csv("combined_v2.csv", index=False)